In [1]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, models
import faiss

In [2]:
# Read Data
data = pd.read_csv('../quran_id_full.csv',memory_map=True)
df_quran = data[['id', 'suraId', 'verseID', 'indoText']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6236 entries, 0 to 6235
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6236 non-null   int64 
 1   suraId    6236 non-null   int64 
 2   verseID   6236 non-null   int64 
 3   ayahText  6236 non-null   object
 4   indoText  6236 non-null   object
 5   readText  6236 non-null   object
dtypes: int64(3), object(3)
memory usage: 292.4+ KB


In [3]:
# store embeded doc to faiss
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')
encoded_data = model.encode(df_quran.indoText.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df_quran))))

In [6]:
encoded_data

array([[-0.17225361,  0.10945583, -0.46310848, ...,  0.3302534 ,
        -0.1409751 , -0.13902146],
       [ 0.11264055, -0.2952658 , -0.18785596, ...,  0.33685857,
        -0.2131397 ,  0.08843155],
       [-0.02374077, -0.07182273, -0.6013576 , ...,  0.38883138,
        -0.55693614, -0.30044872],
       ...,
       [-0.19457728, -0.2955266 , -0.18736142, ...,  0.23071817,
        -0.4615962 ,  0.12744915],
       [-0.0282119 , -0.5524251 , -0.2261983 , ...,  0.46804148,
        -0.30935183, -0.195752  ],
       [-0.01314909,  0.01977904,  0.03076741, ...,  0.21428663,
        -0.29852456,  0.3217204 ]], shape=(6236, 768), dtype=float32)

In [4]:
def fetch_ayat_info(dataframe_idx):
    info = df_quran.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['no_surat'] = info['suraId']
    meta_dict['no_ayat'] = info['verseID']
    meta_dict['teks'] = info['indoText']
    return meta_dict
    
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print(top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_ayat_info(idx) for idx in top_k_ids]
    return results

In [5]:
query="apa itu tuhan yang maha esa"
results=search(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)

(array([[55.207024, 51.177605, 51.139008, 50.94593 , 50.690063]],
      dtype=float32), array([[1922, 1739, 1635, 3527, 3319]]))
>>>> Results in Total Time: 0.11294126510620117


	 {'no_surat': np.int64(12), 'no_ayat': np.int64(40), 'teks': 'Apa yang kamu sembah selain Dia, hanyalah nama-nama yang kamu buat-buat baik oleh kamu sendiri maupun oleh nenek moyangmu. Allah tidak menurunkan suatu keterangan pun tentang hal (nama-nama) itu. Keputusan itu hanyalah milik Allah. Dia telah memerintahkan agar kamu tidak menyembah selain Dia. Itulah agama yang lurus, tetapi kebanyakan manusia tidak mengetahui.'}
	 {'no_surat': np.int64(13), 'no_ayat': np.int64(33), 'teks': 'Maka apakah Tuhan yang menjaga setiap jiwa terhadap apa yang diperbuatnya (sama dengan yang lain)? Mereka menjadikan sekutu-sekutu bagi Allah. Katakanlah, “Sebutkanlah sifat-sifat mereka itu.” Atau apakah kamu hendak memberitahukan kepada Allah apa yang tidak diketahui-Nya di bumi, atau (mengatakan tentang hal itu) sekedar perka